## Multilayer Neural Network

### 1-D case

Library import

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn import datasets
session = tf.Session()
%config Completer.use_jedi = False

Dataset creation

In [5]:
data_size = 25
data_1d = np.random.normal(size = data_size)

- Tensorflow variables

In [6]:
x_input_1d  = tf.placeholder(shape = [data_size], dtype= tf.float32)

Convolutional layer. The following function took a 1-d array and a given filter to return the convolution of both values. TF convolution function receive as an input data an 4-d array, so before feed this function we neeed to expand dimentions of the 1-d given array.

In [10]:
def convolutional_layer_1d(input_1d, my_filter):
    input_2d = tf.expand_dims(input_1d, 0)
    input_3d = tf.expand_dims(input_2d, 0)
    input_4d = tf.expand_dims(input_3d, 3)
    convolution = tf.nn.conv2d(input_4d, filter = my_filter, strides = [1,1,1,1], padding = 'VALID')
    output = tf.squeeze(convolution)
    return(output)

Convolution. First we define the filter using a random distribution.

In [11]:
my_filter = tf.Variable(tf.random_normal(shape = [1, 5, 1, 1]))
my_conv_output = convolutional_layer_1d(x_input_1d, my_filter)

Activation function using ReLu

In [14]:
def activation(input_1d):
    return tf.nn.relu(input_1d)

my_activation_output = activation(my_conv_output)

Max pooling layer. This function return the pooling using a given windom of size $width$. In this case the idea is to take the output of the activation function and return a small-size tensor with the most relevant data of the original output.

In [17]:
def max_pool(input_1d, width):
    input_2d = tf.expand_dims(input_1d, 0)
    input_3d = tf.expand_dims(input_2d, 0)
    input_4d = tf.expand_dims(input_3d, 3)
    pooling = tf.nn.max_pool(input_4d, ksize=[1,1,width, 1], strides = [1,1,1,1], padding = 'VALID')
    output = tf.squeeze(pooling)
    return(output)

my_maxpool_output = max_pool(my_activation_output, width = 5) 

Fully connected layer. This layer convert the maxpooling layer output in to a 1-d array to feed the NN

In [31]:
def fully_connected(input_layer, num_output):
    weight_shape = tf.squeeze(tf.stack([tf.shape(input_layer), [num_output]]))
    weight = tf.random_normal(weight_shape, stddev = 0.1)
    bias = tf.random_normal(shape = [num_output])
    input_layer_2d = tf.expand_dims(input_layer, 0)
    full_output = tf.add(tf.matmul(input_layer_2d, weight), bias)
    full_output_1d = tf.squeeze(full_output)
    return(full_output_1d)

my_full_output = fully_connected(my_maxpool_output, 5)

Variable initializer

In [25]:
init = tf.global_variables_initializer()
session.run(init)

### Function ejecution.
We feed the main functions to see if the result is the expected.

In [32]:
feed_dict = {x_input_1d: data_1d}

#### Convolution
- Using the input data:

In [35]:
data_1d

array([-0.09941728, -0.67428829, -0.04953568,  0.14211985,  1.44992216,
       -0.77852435,  1.0563369 ,  0.79722011,  0.43687146, -1.15331551,
        0.47681568,  0.06056182, -0.07555616,  0.18627288, -1.20940125,
        0.60814375,  0.41341099,  3.10559723, -0.62372542, -0.17035176,
        0.37091793, -0.09960386, -0.55126644, -1.03136013,  0.58911361])

- And the filter:

In [36]:
session.run(my_filter)

array([[[[ 0.89985293]],

        [[ 0.28642622]],

        [[ 0.44073153]],

        [[-0.80505633]],

        [[-0.7751633 ]]]], dtype=float32)

- Convolution result:

In [34]:
print('Input: Size 25, Operation: Convolution with size 5 filter and stride size 1, Result size: 21')
print(session.run(my_conv_output, feed_dict = feed_dict))

Input: Size 25, Operation: Convolution with size 5 filter and stride size 1, Result size: 21
[-1.5427676  -1.1220975   0.44308093 -1.268324    0.56683403  0.4956669
  1.9303097  -0.09659897  0.28274065 -0.9581152   1.2006338   0.6171773
 -1.3577082  -2.650922   -2.7485833   2.668571    0.8362589   2.3194494
  0.06092913  1.1523236   0.4359257 ]


#### Activation function
- Using the convolution result as input data (size = 21)

In [38]:
print(session.run(my_conv_output, feed_dict=feed_dict))

[-1.5427676  -1.1220975   0.44308093 -1.268324    0.56683403  0.4956669
  1.9303097  -0.09659897  0.28274065 -0.9581152   1.2006338   0.6171773
 -1.3577082  -2.650922   -2.7485833   2.668571    0.8362589   2.3194494
  0.06092913  1.1523236   0.4359257 ]


- Activation function result:

In [39]:
print(session.run(my_activation_output, feed_dict = feed_dict))

[0.         0.         0.44308093 0.         0.56683403 0.4956669
 1.9303097  0.         0.28274065 0.         1.2006338  0.6171773
 0.         0.         0.         2.668571   0.8362589  2.3194494
 0.06092913 1.1523236  0.4359257 ]


#### Max Pooling
- Using the activaction function result as input data (size = 21). 
- Max pooling result:

In [40]:
print('Input size = 21, Operation = MaxPooling with window size:5 and stride size:1, Result size: 17')
print(session.run(my_maxpool_output, feed_dict = feed_dict))

Input size = 21, Operation = MaxPooling with window size:5 and stride size:1, Result size: 17
[0.56683403 0.56683403 1.9303097  1.9303097  1.9303097  1.9303097
 1.9303097  1.2006338  1.2006338  1.2006338  1.2006338  2.668571
 2.668571   2.668571   2.668571   2.668571   2.3194494 ]


#### Fully connected layer
- Using the maxpooling result as input data (size: 17)
- Fully connected layer result:

In [41]:
print('Input size = 21, Operation = Fully connected conversion with output size:5, Result size: 5')
print(session.run(my_full_output, feed_dict = feed_dict))

Input size = 21, Operation = Fully connected conversion with output size:5, Result size: 5
[-0.25489312 -0.794742   -2.1049075  -0.07278401 -1.6144413 ]


### 2-D case

We're going to replicate all the operations but in this case using a 2-D input data.

In [43]:
session = tf.Session()   #Restart the session.

Dataset creation

In [45]:
data_size = [10, 10]
data_2d = np.random.normal(size = data_size)
x_input_2d = tf.placeholder(shape = data_size, dtype = tf.float32)

#### Convolution

In [46]:
def conv_layer_2d(input_2d, my_filter):
    input_3d = tf.expand_dims(input_2d, 0)
    input_4d = tf.expand_dims(input_3d, 3)
    convolution = tf.nn.conv2d(input_4d, filter = my_filter, strides = [1,2,2,1], padding = 'VALID')
    output = tf.squeeze(convolution)
    
    return(output)
my_filter = tf.Variable(tf.random_normal(shape = [2,2,1,1]))
my_conv_output = conv_layer_2d(x_input_2d, my_filter)

#### Activation Function

In [47]:
def activation_2d(input_2d):
    return(tf.nn.relu(input_2d))
my_activation_output = activation_2d(my_conv_output)

#### Max pooling layer

In [50]:
def max_pool_2d(input_2d, widht, height):
    input_3d = tf.expand_dims(input_2d, 0)
    input_4d = tf.expand_dims(input_3d, 3)
    pooling = tf.nn.max_pool(input_4d, ksize=[1, height, widht, 1], strides = [1,1,1,1], padding = 'VALID')
    output = tf.squeeze(pooling)
    return(output)
my_maxpool_output = max_pool_2d(my_activation_output, widht = 2, height = 2)

#### Fully connected layer

In [52]:
def fully_connected_2d(input_layer, num_output):
    flat_input = tf.reshape(input_layer, [-1])
    weight_shape = tf.squeeze(tf.stack([tf.shape(flat_input), [num_output]]))
    weight = tf.random_normal(weight_shape, stddev = 0.1)
    bias = tf.random_normal(shape = [num_output])
    input_layer_2d = tf.expand_dims(flat_input, 0)
    full_output = tf.add(tf.matmul(input_layer_2d, weight), bias)
    full_output_1d = tf.squeeze(full_output)
    return(full_output_1d)
my_full_output = fully_connected_2d(my_maxpool_output, 5)

### Functions ejecutions

In [53]:
init = tf.global_variables_initializer()
session.run(init)

In [54]:
feed_dict = {x_input_2d: data_2d}

#### Convolution
- Input data:

In [55]:
data_2d

array([[ 6.69794141e-01, -2.25160742e-01, -8.24612638e-01,
         5.79559523e-01, -1.14428570e+00, -3.52926814e-01,
        -1.80544371e+00,  8.40501244e-01,  2.98406345e-01,
         2.77286718e-01],
       [ 7.08391718e-02,  5.35104511e-01,  1.19381926e+00,
        -3.43527567e-01,  1.96870853e-01,  1.61432893e-01,
        -1.42643910e-01,  4.95835046e-01,  1.89082610e-02,
         1.22827731e+00],
       [ 8.51439820e-01,  3.94907971e-01,  4.63162178e-01,
         3.99733083e-02, -4.66718407e-01,  6.52422735e-01,
         2.11419782e-01, -2.82728140e-01,  1.05665841e-01,
         5.67966292e-01],
       [ 5.30940216e-01,  8.07084817e-01, -1.80624890e+00,
        -5.50062741e-01,  1.86615024e-01, -1.29533443e+00,
         1.16834706e+00, -1.83152883e-01, -8.28473859e-02,
         2.94814013e-01],
       [ 2.92139126e-01, -1.64149137e-01, -1.64592971e-01,
        -1.84087750e-01,  7.17392022e-01, -1.66528420e+00,
         1.19868783e-01,  2.20309968e+00,  1.53871607e+00,
         2.

- Filter to use:

In [56]:
print(session.run(my_filter))

[[[[ 0.05529318]]

  [[-0.8814632 ]]]


 [[[ 0.27901518]]

  [[ 0.9101391 ]]]]


- Convolution operation:

In [57]:
print('Input: Size [10,10], Operation: Convolution with size: [2,2] filter and stride size: [2,2], Result size: [5,5]')
print(session.run(my_conv_output, feed_dict = feed_dict))

Input: Size [10,10], Operation: Convolution with size: [2,2] filter and stride size: [2,2], Result size: [5,5]
[[ 0.7422907  -0.5360199   0.44967714 -0.42922068  0.8952607 ]
 [ 0.5816817  -1.0142298  -1.7277591   0.42019647 -0.24959272]
 [-0.8116984  -1.4624821   1.4557121  -2.9640646   0.23206139]
 [ 0.5069617  -2.4954777   0.28051472  0.67162883  0.9816976 ]
 [ 0.320952    0.716101   -0.78223175  0.2568572   1.2023238 ]]


#### Activation Function

In [59]:
print('Input size: [5,5], Operation: ReLu to convolution output, Result size: [5,5]')
print(session.run(my_activation_output, feed_dict = feed_dict))

Input size: [5,5], Operation: ReLu to convolution output, Result size: [5,5]
[[0.7422907  0.         0.44967714 0.         0.8952607 ]
 [0.5816817  0.         0.         0.42019647 0.        ]
 [0.         0.         1.4557121  0.         0.23206139]
 [0.5069617  0.         0.28051472 0.67162883 0.9816976 ]
 [0.320952   0.716101   0.         0.2568572  1.2023238 ]]


#### Max Pooling

In [60]:
print('Input size: [5, 5], Operation = MaxPooling with window size:[2,2] and stride size:1, Result size: [4,4]')
print(session.run(my_maxpool_output, feed_dict = feed_dict))

Input size: [5, 5], Operation = MaxPooling with window size:[2,2] and stride size:1, Result size: [4,4]
[[0.7422907  0.44967714 0.44967714 0.8952607 ]
 [0.5816817  1.4557121  1.4557121  0.42019647]
 [0.5069617  1.4557121  1.4557121  0.9816976 ]
 [0.716101   0.716101   0.67162883 1.2023238 ]]


#### Fully connected layer

In [61]:
print('Input size:[4,4], Operation = Fully connected conversion with output size:5, Result size: 5')
print(session.run(my_full_output, feed_dict = feed_dict))

Input size:[4,4], Operation = Fully connected conversion with output size:5, Result size: 5
[ 1.0684043   0.2986397  -0.06971306 -0.34995604  0.52973837]
